# Rapport TP2

<!-- TODO: ajouter les noms et matricules -->
Xavier Jouslin - 2073104

Haroun Mili - 2144744

**Note: to be able to run the code**
```bash
pip install scikit-image
```

In [39]:
!pip install scikit-image
!pip install einops



## Question 1

### Description generale de l'algorithme de Karhunen-Loève

Soit une image composée de plusieurs bases (par exemple, RGB ou YUV). Ces représentations sont conformes et normées mais pas forcément les plus optimisées. L'algorithme de Karhunen-Loève permet de trouver la base avec le plus d'information possible en utilisant le moins de bases possible. Autrement dit, il faut trouver l'entropie maximale pour chaque composante choisie de manière à ce que le nombre de composantes reste minimal. 
### Etape de l'algorithme de Karhunen-Loève

Le principe général de la transformation de Karhunen-Loève pour les images est de trouver une base qui permette de concentrer l'information la plus essentielle de l'image (information clé pour la distinction de l'image) et de la condenser (tel que tiré du cours INF8770). Cela s'effectue en faisant la moyenne des composantes R, G, B des pixels, puis en calculant la covariance de chacun des pixels. Lorsque cela est fait, on trouve les vecteurs propres de la matrice des covariances obtenue précédemment. Ensuite, on multiplie cette matrice par les vecteurs propres.
On exprime chaque pixel de l'image dans une nouvelle base dans laquelle on ne garde que les vecteurs propres ayant le plus d'énergie. On pourrait ainsi mettre à zéro les vecteurs propres de faibles énergies. Lorsque cela est fait, on effectue une projection des données des images originales sur l'espace des composantes principales, ce qui réduit la dimensionnalité de la représentation.

### Difference avec les autres methodes de transformation

#### Discrete Cosine Transform

La transforme en cosinus discret utilise le principe de frequence de cosinus. Cette transformation est moins optimal, plus rapide que l'algorithme de Karhunen-Loève. L'avantage avec cette algorithme est qu'il est parrallelisable.La DCT, ne fait qu’utiliser une base fixe de cosinus fixe. On ne regarde pas la variance, on fait juste directement appliquer la fonction cosinus avec des paramètres associés. C’est généralement utile pour représenter les données ayant des signaux périodiques. 


#### Odelette

l'algorithme de Odelette utilise le principe des ondes a la difference des cosinus avec des frequence differentes pour emcode les images. Cette algorithme est plus rapide mais moins optimal que la transforme de Karhunen-Loève. 
Les ondelettes quant à elles utilisent des fonctions localisées dans le temps et dans la fréquence afin de filtrer l’image. Elles utilisent du filtrage passe-bas et passe-haut en cascade. Pour traiter d’autres bandes de fréquence, ils font du rééchantillonnage.  Les ondelettes quant à elles sont utiles pour représenter les signaux non périodiques comme ils seraient concentrés dans quelques coefficients seulement plutôt que distribués partout dans l’image de manière périodique comme la DCT. Il n’y a pas de différenciation de variance comme dans la transformée de Karhunen-Loève, et pas de choix relatif aux pixels de l’image mais plutôt aux caractéristiques spécifiques dans l’image. 

Il est important de soulever le fait que Karhunen-Loève est le mieux placé pour être utilisé à des fins générales avec différents types de données, tandis que DCT est plus utile pour des données avec des signaux périodiques et que les ondelettes sont plus utiles pour les données ayant peu de périodicité. 



## Question 2

### Description de la solution

ImageKL est l'implémentation de la compression d'image avec l'algorithme de Karhunen-Loève. Chaque image est transformée en matrice 2D (taille de l'image * canal colorimétrique). La méthode d'encodage permet de compresser l'image avec ou sans la possibilité de changer de base colorimétrique.

In [45]:
from os import path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from einops import rearrange

class ImageKL:
    def __init__(self, path_code: str):
        self.__width = 0
        self.__height = 0
        self.__quantifier = ()
        self.__compress_code = None
        self.code = self.tilde_image(path_code)

    def tilde_image(self, image_path: str):
        image = plt.imread(image_path)
        self.__width = image.shape[1]
        self.__height = image.shape[0]
        return rearrange(image, 'h w c -> (h w) c')

    def psnr(self):
        return 10 * np.log10(self.__mse()) + 20 * np.log10(self.__msi())

    def ssim(self):
        covariance_images = np.cov(np.concatenate((np.reshape(self.code, (-1, 1)), np.reshape(self.__compress_code, (-1, 1))), axis=1).T)[0,1]
        variance_base = np.cov(np.reshape(self.code, (-1, 1)).T)
        variance_compressed = np.cov(np.reshape(self.__compress_code, (-1, 1)).T)
        return (2 * np.mean(self.code) * np.mean(self.__compress_code) + 0.01) * (2 * covariance_images + 0.03) / ((np.mean(self.code) ** 2 + np.mean(self.__compress_code) ** 2 + 0.01) * (variance_base + variance_compressed + 0.03))

    def compression_ratio(self):
        code_color_space = np.power(2, 8 * 3)
        compress_code_color_space = np.power(2, self.__quantifier[0]) * np.power(2, self.__quantifier[1]) * np.power(2, self.__quantifier[2])
        code_length = np.array(self.code).shape[0] * np.array(self.code).shape[1]
        compress_code = np.array(self.__compress_code).shape[0] * np.array(self.__compress_code).shape[1]
        division1=compress_code/code_length
        division2=compress_code_color_space/code_color_space
        return division1*division2

    def rgb2yuv(self, rgb):
        return rearrange(cv2.cvtColor(rearrange(rgb, '(h w) c -> h w c', h=self.__height, w=self.__width).astype(np.float32), cv2.COLOR_RGB2YUV), 'h w c -> (h w) c')

    def yuv2rgb(self, yuv):
        return rearrange(cv2.cvtColor(rearrange(yuv, '(h w) c -> h w c', h=self.__height, w=self.__width).astype(np.float32), cv2.COLOR_YUV2RGB), 'h w c -> (h w) c')
        

    def encode(self, quantifier=(8, 8, 8), is_yuv=False):
        if is_yuv:
            self.code = self.rgb2yuv(self.code)
        kl_transform = self.kl_transform()
        self.__quantifier = quantifier
        self.__compress_code = self.kl_transform_inv(self.reconstruct(self.quantify(quantifier, kl_transform), kl_transform))
        if is_yuv:
            self.__compress_code = self.yuv2rgb(self.__compress_code)
        return rearrange(np.clip(self.__compress_code, 0, 1), '(h w) c -> h w c', h=self.__height, w=self.__width)

    def __average(self, code):
        mean = np.mean(code, axis=0)
        average = np.zeros(code.shape)
        for i in range(code.shape[1]):
            average[:, i] = mean[i]
        return average

    def __covariant(self, code):
        return np.cov(code, rowvar=False)

    def __eigenvector(self, code):
        return np.linalg.eig(self.__covariant(code))[1]

    def __inverse_eigenvector(self, code):
        return np.linalg.inv(self.__eigenvector(code))

    def kl_transform(self):
        return np.transpose(np.matmul(self.__eigenvector(self.code), np.transpose(np.subtract(self.code, self.__average(self.code)))))

    def quantify(self, quantifier, kl_transform_outpout):
        canals = [kl_transform_outpout[:, i] for i in range(3)]
        return  [np.linspace(np.min(canals[i]), np.max(canals[i]), num=2**quantifier[i]) for i in range(len(canals))]

    def reconstruct(self, quantifier, kl_transform_outpout):
        canals = [kl_transform_outpout[:, i] for i in range(3)]
        compress_canals = [[quantifier[i][np.abs(quantifier[i] - data).argmin()] for data in canals[i]] for i in range(len(quantifier))]
        return np.transpose(np.array(compress_canals))

    def kl_transform_inv(self, kl_transform):
        return np.transpose(np.matmul(np.transpose(self.__inverse_eigenvector(self.code)), np.transpose(np.add(kl_transform, self.__average(self.code)))))

    def __msi(self):
        return max(np.power(2, self.__quantifier))

    def __mse(self):
        return 1 / np.mean((self.__compress_code - self.code) ** 2)


In [48]:
import pandas as pd
import os

notebook_path = os.path.abspath("rapport.ipynb")
script_path = os.path.dirname(notebook_path)
image_files = [os.path.join(script_path, "data", f) for f in os.listdir(os.path.join(script_path, "data")) if os.path.isfile(os.path.join(script_path, "data", f))]
quantifiers = [(8, 8, 8), (8, 8, 4), (8, 8, 0), (8, 4, 0)]
result = pd.DataFrame(columns=["image", "canal", "is_yuv", "psnr", "ssim", "compression_ratio"])
for image_file in image_files:
    image = ImageKL(image_file)
    for quantifier in quantifiers:
        for is_yuv in [True, False]:
            image.encode(quantifier, is_yuv)
            image_result = {"image": [path.basename(image_file)], "canal": [", ".join([str(canal) for canal in quantifier])], "is_yuv": [is_yuv], "psnr": [image.psnr()], "ssim": [image.ssim()], "compression_ratio": [image.compression_ratio()]}
            image_result = pd.DataFrame(image_result)
            result = pd.concat([result, image_result], ignore_index=True)
print(result)

          image    canal is_yuv       psnr      ssim  compression_ratio
0   kodim01.png  8, 8, 8   True  44.417300  0.005286           1.000000
1   kodim01.png  8, 8, 8  False  49.583820 -0.059124           1.000000
2   kodim01.png  8, 8, 4   True  48.223620 -0.188806           0.062500
3   kodim01.png  8, 8, 4  False  53.828507  0.126648           0.062500
4   kodim01.png  8, 8, 0   True  56.909139  0.231443           0.003906
5   kodim01.png  8, 8, 0  False  60.995416  0.438610           0.003906
6   kodim01.png  8, 4, 0   True  48.299996 -0.370992           0.000244
7   kodim01.png  8, 4, 0  False  52.742542  0.065942           0.000244
8   kodim02.png  8, 8, 8   True  59.746583  0.417844           1.000000
9   kodim02.png  8, 8, 8  False  68.281711  0.892883           1.000000
10  kodim02.png  8, 8, 4   True  47.136916 -0.277212           0.062500
11  kodim02.png  8, 8, 4  False  52.697260  0.080126           0.062500
12  kodim02.png  8, 8, 0   True  45.637899 -0.138319           0

## Question 3

Nous allons trie le tableau que nous avons obtenu dans la section precedente de telle facon d'optenir le meilleure modele ayant le meilleure resultat.

In [47]:
best_result = result.sort_values(by=["image", "psnr", "ssim", "compression_ratio"], ascending=[True, False, False, True])
best_result = best_result.groupby("image").head(1)
print(best_result)

          image    canal is_yuv       psnr      ssim  compression_ratio
5   kodim01.png  8, 8, 0  False  60.995416  0.438610           0.003906
9   kodim02.png  8, 8, 8  False  68.281711  0.892883           1.000000
23  kodim05.png  8, 4, 0  False  55.523333  0.229688           0.000244
25  kodim13.png  8, 8, 8  False  59.702867  0.519967           1.000000
39  kodim23.png  8, 4, 0  False  55.714587  0.217189           0.000244


Pour rappel: les canaux de RGB représentent le rouge, vert et bleu tandis que red, blue and green and YUV stands for (Y) luma, or brightness, (U) blue projection and (V) red projectionOn observe que les meilleures resultat se trouve avec des quantificateurs(canal) possedant une tres forte etalement dans l espace du rouge et du vert. Dans les echantillons pris dans les donnees, il y a preponderance de rouge et de vert. On observe aussi que la base colorimetrique dans nos meilleures resultats correspond au RGB plutot que YUV. YUV devrait permettre une meilleure compression. la matrice KL de l'image 
YUV permet de differencie la luminosite de l'image versus l'information de la couleur de l'image.

## Question 4

In [43]:
class ImageKLFeat:
    def __init__(self, path_code_fit_image: str, path_code_test_image: str):
        self.__compress_code = None
        self.__quantifier = ()
        self.code = ImageKL(path_code_fit_image)
        self.test_code = ImageKL(path_code_test_image)

    def psnr(self):
        return 10 * np.log10(self.__mse()) + 20 * np.log10(self.__msi())

    def ssim(self):
        covariance_images = np.cov(np.concatenate((np.reshape(self.test_code.code, (-1, 1)), np.reshape(self.__compress_code, (-1, 1))), axis=1).T)[0,1]
        variance_base = np.cov(np.reshape(self.test_code.code, (-1, 1)).T)
        variance_compressed = np.cov(np.reshape(self.__compress_code, (-1, 1)).T)
        return (2 * np.mean(self.test_code.code) * np.mean(self.__compress_code) + 0.01) * (2 * covariance_images + 0.03) / ((np.mean(self.test_code.code) ** 2 + np.mean(self.__compress_code) ** 2 + 0.01) * (variance_base + variance_compressed + 0.03))

    def encode(self, quantifier=(8, 8, 8), is_yuv=False):
        self.__quantifier = quantifier
        if is_yuv:
            self.code = self.rgb2yuv(self.code)
            self.test_code = self.rgb2yuv(self.test_code)
        kl_transform = self.code.kl_transform()
        self.__compress_code = self.test_code.kl_transform_inv(self.test_code.reconstruct(self.code.quantify(quantifier, kl_transform), kl_transform))
        return self.__compress_code
    
    def __msi(self):
        return max(np.power(2, self.__quantifier))

    def __mse(self):
        return 1 / np.mean((self.__compress_code - self.test_code.code) ** 2)


In [44]:
compute_image_files = [[(f, f1) for f1 in image_files if f1 != f] for f in image_files]
result = pd.DataFrame(columns=["image", "canal", "is_yuv", "psnr", "ssim"])
for image_file in compute_image_files:
    for combination in image_file:        
        image = ImageKLFeat(combination[0], combination[1])
        best_colorimetric_base = best_result.loc[best_result["image"] == path.basename(combination[0])]["is_yuv"].values[0]
        best_quantifier = [int(quant) for quant in best_result.loc[best_result["image"] == path.basename(combination[1])]["canal"].values[0].split(", ")]
        image.encode(quantifier, best_colorimetric_base)
        image_result = {"image": ", ".join([path.basename(combination[0]), path.basename(combination[1])]), "canal": [", ".join([str(canal) for canal in quantifier])], "is_yuv": [is_yuv], "psnr": [image.psnr()], "ssim": [image.ssim()]}
        image_result = pd.DataFrame(image_result)
        result = pd.concat([result, image_result], ignore_index=True)
print(result)

                       image    canal is_yuv       psnr      ssim
0   kodim01.png, kodim02.png  8, 4, 0  False  61.790258  0.692596
1   kodim01.png, kodim05.png  8, 4, 0  False  52.357393 -0.082803
2   kodim01.png, kodim13.png  8, 4, 0  False  50.807518 -0.226098
3   kodim01.png, kodim23.png  8, 4, 0  False  50.224957  0.049084
4   kodim02.png, kodim01.png  8, 4, 0  False  59.292190  0.397960
5   kodim02.png, kodim05.png  8, 4, 0  False  52.579081 -0.102978
6   kodim02.png, kodim13.png  8, 4, 0  False  50.786554 -0.176172
7   kodim02.png, kodim23.png  8, 4, 0  False  50.295179  0.064563
8   kodim05.png, kodim01.png  8, 4, 0  False  57.748331  0.331800
9   kodim05.png, kodim02.png  8, 4, 0  False  58.679060  0.531947
10  kodim05.png, kodim13.png  8, 4, 0  False  50.638600 -0.203653
11  kodim05.png, kodim23.png  8, 4, 0  False  50.115935 -0.008152
12  kodim13.png, kodim01.png  8, 4, 0  False  56.783313  0.332908
13  kodim13.png, kodim02.png  8, 4, 0  False  56.529369  0.420328
14  kodim1

## Question 5

On observe selon nos resultats que la qualite de la compression est deteriore. Il est plus interressant d'utilise la meme image pour obtenir la matrice KL et la matrice de quantification.
En guise de rappel "PSNR (sigle de Peak Signal to Noise Ratio) est une mesure de distorsion utilisée en image numérique, tout particulièrement en compression d'image. Elle permet de quantifier la performance des codeurs en mesurant la qualité de reconstruction de l'image compressée par rapport à l'image originale." d'après Wikipédia. Tandis que "l'idée de SSIM est de mesurer la similarité de structure entre les deux images, plutôt qu'une différence pixel à pixel comme le fait par exemple le PSNR. L'hypothèse sous-jacente est que l'œil humain est plus sensible aux changements dans la structure de l'image."(Wikipédia). 
Pour interprêter les valeurs de l'index SSIM, il faut comprendre que "The resultant SSIM index is a decimal value between -1 and 1, where 1 indicates perfect similarity, 0 indicates no similarity, and -1 indicates perfect anti-correlation." (Wikipédia) 
Ainsi, pour la première image et seconde image par exemple, nous remarquons que la mesure de psnr est élevée à 61%. On remarque que c'est normal puisque l'on a gardé les canaux Rouge et Verts de la première image pour construire la seconde. Comme la seconde image est majoritairement constituée de rouge, alors il est normal qu'il y eut correspondance vis-à-vis de la couleur des pixels (calculé par le critère psnr.) Maintenant étant donné que structurellement, on avait des formes géométriques rectangulaires dans les 2 images (à cause de la forme des fenêtres, portes et des planches de bois) et que ces formes qui établissent la structure sont très fortes en rouge, alors il est aussi normal qu'on avait corrélation structurelle entre ces 2 images (la délimitation de la structure est facilitée par les canaux retenus 8:4:0, et donc par les couleurs rouges). 

    
Pour l'image 5, on remarque que ses valeurs de SSIM les plus hautes sont avec les images 1 et 2. Les raisons peuvent être dues aux symboles répétés, ici les motos, qui structurellement sont similaires à aux briques et fenêtres répétées de "kodim01", et aux morceaux de bois répétés de "kodim02". On remarque pour ces mêmes images une bonne corrélation de PSNR dues aux couleurs de rouge et vert et blanc prédomianntes dans les images "kodim02" (rouge et blanc) et "kodim01" (rouge et blanc verdatre) et aux motos dont des parties relativement géométriques étaient rouges. 

Pour l'image "kodim13", on remarque que les couleurs prédominantes sont le vert, le marron/rouge. On peut donc directement prédire qu'il y a une bonne PSNR avec "kodim05" et "kodim01"
On remarque pour cette image qu'il y a profondeur et pas réellement d'élément au premier plan. C'est une vue plutôt scénique. C'est cette vue scénique qui tend vers un plan plat qui expliquerait les faibles valeurs structurelles avec les autres images. 

Globalement c'est la première et la 2ème image qui sont structurellement les plus faciles à construire. On peut déduire que c'est probablement parce que les autres images de toutes les manières se font retirer complètement le canal Bleu et partiellement le canal Vert, alors que les images "kodim01" et "kodim02" n'ont de manière prédominante que du Rouge ("kodim02") et un peu de blanc tendant vers un vert ("kodim01"). Ainsi, l'espace des couleurs original a peu de chances de compliquer le décompression des images "kodim01" et "kodim02".

L'image "kodim13" est la plus compliquée à avoir un bon SSIM lorsqu'on prend en compte les énergies par la transformée de KL des autres images parce que ses couleurs ne comportent pas de rouge de manière aussi intense que les autres images. Elle possède du bleu et du vert au contraire. On remarque aussi que l'image qui a le plus du mal à avoir un bon PSNR est l'image "kodim23, ce qui peut s'expliquer par le fait qu'elle possède du vert, mais aussi du jaune (qui est combinaison de rouge et vert) et du bleu. On remarque ici des couleurs peu présentes dans les autres images. Ainsi il est normal que la transformée de KL qui de base nullifie le canal bleu et une partie du canal vert, et qui en plus est appliquée sur des images qui ont très peu de bleu et moins de vert que de rouge fasse en sorte que la similtude des couleurs entre l'image originale et finale soit moins bonne. Cette même absence de rouge explique aussi pourquoi structurellement elle est très faible en prenant le résultat de la KL des autres images puisque ses délimitations structurelles sont surtout opérées par le vert, qui est divisé par un facteur 15 à cause de 8:4:0. 

